In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import shutil
from pymatgen.io.vasp.inputs import Poscar
from pymatgen.core.structure import Structure

def add_selective_dynamics_to_slab(slab, fixed_fraction=0.1):
    z_coords = [site.coords[2] for site in slab]
    z_min = min(z_coords)
    z_max = max(z_coords)
    threshold = z_min + fixed_fraction * (z_max - z_min)
    
    sd_flags = []
    for site in slab:
        if site.coords[2] <= threshold:
            sd_flags.append([False, False, False])
        else:
            sd_flags.append([True, True, True])
    slab.add_site_property("selective_dynamics", sd_flags)
    return slab

def process_single_vasp_file(file_path, output_path, fixed_fraction=0.1):
    try:
        poscar = Poscar.from_file(file_path)
        slab = poscar.structure
        slab_fixed = add_selective_dynamics_to_slab(slab, fixed_fraction=fixed_fraction)
        
        # 新文件路径为对应子文件夹下的 POSCAR
        poscar_fixed = Poscar(slab_fixed)
        poscar_fixed.write_file(output_path)
        return slab_fixed, output_path
    except Exception as e:
        print(f"[ERROR] 处理文件 {file_path} 时出错：{e}")
        return None, None

def main():
    # 定义主 fix 文件夹
    cwd = os.getcwd()
    master_fix_folder = os.path.join(cwd, "fix")
    os.makedirs(master_fix_folder, exist_ok=True)
    
    # 遍历当前工作目录下的所有模型文件夹
    for item in os.listdir(cwd):
        folder_path = os.path.join(cwd, item)
        if os.path.isdir(folder_path) and not item.startswith("fix"):
            files = os.listdir(folder_path)
            best_files = [f for f in files if f.startswith("totalbest") and f.endswith(".vasp")]
            
            if not best_files:
                continue
                
            print(f"\n正在处理模型文件夹：{folder_path}")
            
            # 为每个模型创建对应的子文件夹
            model_fix_folder = os.path.join(master_fix_folder, item)
            os.makedirs(model_fix_folder, exist_ok=True)
            
            for file in best_files:
                original_file = os.path.join(folder_path, file)
                # 新文件路径为子文件夹下的 POSCAR
                new_file_path = os.path.join(model_fix_folder, "POSCAR")
                
                # 拷贝文件到对应子文件夹并重命名
                shutil.copy(original_file, new_file_path)
                print(f"复制文件：{original_file} -> {new_file_path}")
                
                # 对复制后的文件添加 selective dynamics
                slab_fixed, out_file = process_single_vasp_file(new_file_path, new_file_path, fixed_fraction=0.5)
                if out_file:
                    print(f"固定后的文件写入：{out_file}")

if __name__ == "__main__":
    main()


正在处理模型文件夹：c:\Users\21cso\Desktop\Binding_effect\Interaction_effect\1\Hf4O8
复制文件：c:\Users\21cso\Desktop\Binding_effect\Interaction_effect\1\Hf4O8\totalbest_ID1_10-1_47.vasp -> c:\Users\21cso\Desktop\Binding_effect\Interaction_effect\1\fix\Hf4O8\POSCAR
固定后的文件写入：c:\Users\21cso\Desktop\Binding_effect\Interaction_effect\1\fix\Hf4O8\POSCAR

正在处理模型文件夹：c:\Users\21cso\Desktop\Binding_effect\Interaction_effect\1\Nb8O20
复制文件：c:\Users\21cso\Desktop\Binding_effect\Interaction_effect\1\Nb8O20\totalbest_ID2_011_46.vasp -> c:\Users\21cso\Desktop\Binding_effect\Interaction_effect\1\fix\Nb8O20\POSCAR
固定后的文件写入：c:\Users\21cso\Desktop\Binding_effect\Interaction_effect\1\fix\Nb8O20\POSCAR

正在处理模型文件夹：c:\Users\21cso\Desktop\Binding_effect\Interaction_effect\1\O10V4
复制文件：c:\Users\21cso\Desktop\Binding_effect\Interaction_effect\1\O10V4\totalbest_ID6_101_14.vasp -> c:\Users\21cso\Desktop\Binding_effect\Interaction_effect\1\fix\O10V4\POSCAR
固定后的文件写入：c:\Users\21cso\Desktop\Binding_effect\Interaction_effect\1\fix\